# STEP 5: Plot the Veery observations by month

First thing first – let’s load your stored variables and import
libraries.

In [2]:
%store -r

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>In the imports cell, we’ve included some packages that you will need.
Add imports for packages that will help you:</p>
<ol type="1">
<li>Make interactive maps with vector data</li>
</ol></div></div>

In [3]:
# Get month names
import calendar

# Libraries for Dynamic mapping
import cartopy.crs as ccrs

import hvplot.pandas
import panel as pn
import geoviews as gv
import calendar


/opt/conda/lib/python3.11/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### Create a simplified `GeoDataFrame` for plotting

Plotting larger files can be time consuming. The code below will
streamline plotting with `hvplot` by simplifying the geometry,
projecting it to a Mercator projection that is compatible with
`geoviews`, and cropping off areas in the Arctic.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Simplify ecoregion data</div></div><div class="callout-body-container callout-body"><p>Download and save ecoregion boundaries from the EPA:</p>
<ol type="1">
<li>Simplify the ecoregions with <code>.simplify(.05)</code>, and save
it back to the <code>geometry</code> column.</li>
<li>Change the Coordinate Reference System (CRS) to Mercator with
<code>.to_crs(ccrs.Mercator())</code></li>
<li>Use the plotting code that is already in the cell to check that the
plotting runs quickly (less than a minute) and looks the way you want,
making sure to change <code>gdf</code> to YOUR <code>GeoDataFrame</code>
name.</li>
</ol></div></div>

In [4]:
# Simplify the geometry to speed up processing
eco_gdf.geometry=eco_gdf.simplify(.05,preserve_topology=False)
# Change the CRS to Mercator for mapping
eco_gdf=eco_gdf.to_crs(ccrs.Mercator())
gbif_gdf=gbif_gdf.to_crs(ccrs.Mercator())
# Check that the plot runs in a reasonable amount of time
eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())
eco_gdf

,name,area,geometry
ecoregion_id,,,
1.0,Adelie Land tundra,0.038948,"POLYGON ((17090104.083 -10646152.188, 17100906..."
2.0,Admiralty Islands lowland rain forests,0.170599,"MULTIPOLYGON (((16461836.873 -254248.259, 1644..."
3.0,Aegean and Western Turkey sclerophyllous and m...,13.844952,"MULTIPOLYGON (((3391149.749 4336064.109, 33985..."
4.0,Afghan Mountains semi-desert,1.355536,"MULTIPOLYGON (((7369001.698 4093509.259, 73168..."
5.0,Ahklun and Kilbuck Upland Tundra,8.196573,"MULTIPOLYGON (((-17912546.29 8050475.591, -179..."
...,...,...,...
848.0,Sulawesi lowland rain forests,9.422097,"MULTIPOLYGON (((14113374.546 501721.962, 14128..."
212.0,East African montane forests,5.010930,"MULTIPOLYGON (((4275410.576 -149750.128, 42781..."
224.0,Eastern Arc forests,0.890325,"MULTIPOLYGON (((4302962.15 -537675.265, 431112..."


<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Map migration over time</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>If applicable, replace any variable names with the names you defined
previously.</li>
<li>Replace <code>column_name_used_for_ecoregion_color</code> and
<code>column_name_used_for_slider</code> with the column names you wish
to use.</li>
<li>Customize your plot with your choice of title, tile source, color
map, and size.</li>
</ol>
<div data-__quarto_custom="true" data-__quarto_custom_type="Callout"
data-__quarto_custom_context="Block" data-__quarto_custom_id="3">
<div data-__quarto_custom_scaffold="true">

</div>
<div data-__quarto_custom_scaffold="true">
<p>Your plot will probably still change months very slowly in your
Jupyter notebook, because it calculates each month’s plot as needed.
Open up the saved HTML file to see faster performance!</p>
</div>
</div></div></div>

In [5]:
#occurrence_df
gbif_gdf
gbif_gdf.hvplot(geo=True, crs=ccrs.Mercator())


:Points   [Longitude,Latitude]

In [6]:
# Join the occurrences with the plotting GeoDataFrame
occurrence_gdf = eco_gdf.join(occurrence_df)
# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = occurrence_gdf.total_bounds

# Define the slider widget
slider = pn.widgets.DiscreteSlider(
    name='Month',
    options={calendar.month_name[i]: i for i in range(1, 13)}
)

# Create the interactive migration plot
migration_plot = (
    occurrence_gdf.hvplot(
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        c='norm_occurrences',
        groupby='month',
        width=800, height=600,  # Explicit dimensions to fix W-1005
        colorbar=False,
        widgets={'month': slider},
        widget_location='bottom'
    )
)

# Save the plot as an interactive HTML file
migration_plot.save('migration.html', embed=True)

# Display the plot in a notebook or Panel server
migration_plot

 83%|████████▎ | 10/12 [00:00<00:00, 14.26it/s]

BokehModel(combine_events=True, render_bundle={'docs_json': {'266308dc-69d1-45ff-bc8f-43ea04774b83': {'version…

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?: Fix the month labels</div></div><div class="callout-body-container callout-body"><p>Notice that the <code>month</code> slider displays numbers instead of
the month name. Use <code>pn.widgets.DiscreteSlider()</code> with the
<code>options=</code> parameter set to give the months names. You might
want to try asking ChatGPT how to do this, or look at the documentation
for <code>pn.widgets.DiscreteSlider()</code>. This is pretty tricky!</p></div></div>